<a href="https://colab.research.google.com/github/NSuprotivniy/anti-spoofing/blob/master/nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-gs_7l3uf
  Stored in directory: /tmp/pip-ephem-wheel-cache-nedtndh5/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [2]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, Dropout
from keras_vggface.vggface import VGGFace
from keras.utils import Sequence
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
import os
from glob import glob
import numpy as np
from google.colab import drive

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9759545197744174797, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5301430672090695243
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8332224278160083208
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14800630580
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2065007534234397984
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [5]:
vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3), model='resnet50')

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
vgg_model.trainable = False
last_layer = vgg_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(2048, activation='relu', name='dense_relu')(x)
x = Dropout(0.1, seed=17)(x)
out = Dense(1, activation='sigmoid', name='classifier')(x)
custom_vgg_model = Model(vgg_model.input, out)
custom_vgg_model.compile(optimizer=Adam(lr=0.000001), loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
drive.mount("/content/nsuprotivniy/")
data_path = "/content/nsuprotivniy/My Drive/Colab Notebooks/data/"

Drive already mounted at /content/nsuprotivniy/; to attempt to forcibly remount, call drive.mount("/content/nsuprotivniy/", force_remount=True).


In [0]:
class FaceReplayAttackUQDataset(Sequence):  
  def __init__(self):
    self.live_path = glob(os.path.join(data_path, "splitted/live/*.npy"))
    self.paper_path = glob(os.path.join(data_path, "splitted/paper/*.npy"))
    
  def __len__(self):
    return max(len(self.live_path), len(self.paper_path))


  def __getitem__(self, idx):
    paper = np.load(self.paper_path[idx % len(self.paper_path)]) 
    live = np.load(self.live_path[idx % len(self.live_path)])
    X = np.concatenate((paper, live))
    y = np.concatenate((np.ones(len(paper)), np.zeros(len(live))))
    return X, y

In [0]:
if (not os.path.exists(os.path.join(data_path, "checkpoints"))):
  os.mkdir(os.path.join(data_path, "checkpoints"))
checkpoint = ModelCheckpoint(os.path.join(data_path, "checkpoints/checkpoints"), verbose=1, period=10)
callbacks_list = [checkpoint]

In [0]:
generator = FaceReplayAttackUQDataset()

In [0]:
custom_vgg_model.fit_generator(generator, epochs=1000, verbose=1, callbacks=callbacks_list, shuffle=True)

Epoch 1/1000
2048/2048 [==============================] - 550s 269ms/step - loss: 0.0274 - acc: 0.9907
Epoch 2/1000
2048/2048 [==============================] - 547s 267ms/step - loss: 8.0730e-04 - acc: 0.9998
Epoch 3/1000
2048/2048 [==============================] - 547s 267ms/step - loss: 1.1502e-04 - acc: 1.0000
Epoch 4/1000
2048/2048 [==============================] - 547s 267ms/step - loss: 3.0901e-05 - acc: 1.0000
Epoch 5/1000
1136/2048 [===============>..............] - ETA: 4:03 - loss: 1.3424e-05 - acc: 1.0000

In [15]:
generator.__getitem__(1)[0].shape

(19, 224, 224, 3)

In [0]:
X, y = generator.__getitem__(1)
predicted = custom_vgg_model.predict(X)

In [35]:
np.mean((predicted > 0.5) == y)

0.48931701972242514

In [1]:
295 * 256 * 55 * 55 * 4 / 1024 /1024 /1024

0.8510351181030273